In [16]:

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam
import imageio as io
import matplotlib.pyplot as plt
from vis.visualization import visualize_saliency
from vis.utils import utils
from keras import activations
from vis.visualization import visualize_activation,visualize_cam, overlay
import matplotlib.cm as cm
import os

gpath='/content/gdrive/My Drive/Findit/Death Valley/'

In [18]:
%%time
#load model from disk
from keras.models import model_from_json
# load json and create model
json_file = open(gpath+'modelv7.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights(gpath+'modelv7.h5')
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
print("Loaded model from disk")

Loaded model from disk
CPU times: user 5.55 s, sys: 170 ms, total: 5.72 s
Wall time: 5.69 s


In [19]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

In [0]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

In [21]:
#Classes
train_datagen=ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True) 
generator= train_datagen.flow_from_directory(gpath+'MobileNetData', batch_size=512, seed=1234)
label_map_rev = (generator.class_indices)
label_map=dict((v,k) for k,v in label_map_rev.items())#
label_map

Found 519 images belonging to 49 classes.


{0: 'AMERICAN PIPIT',
 1: 'AMERICAN ROBIN',
 2: 'BENDIRES THRASHER',
 3: 'Beavertail Cactus',
 4: 'CACTUS WREN',
 5: 'Cottontop Barrel Cactus',
 6: 'Desert Bighorn Sheep',
 7: 'EUROPEAN STARLING',
 8: 'Grizzly Bear Pricklypear',
 9: 'HOUSE SPARROW',
 10: 'Joshua Tree',
 11: 'LOGGERHEAD SHRIKE',
 12: 'MOUNTAIN BLUEBIRD',
 13: 'Pickleweed',
 14: 'ROCK WREN',
 15: 'RUBY-CROWNED KINGLET',
 16: 'Silver Cholla cactus',
 17: 'VARIED THRUSH',
 18: 'badger',
 19: 'black-tailed jackrabbit',
 20: 'bobcat',
 21: 'burro',
 22: 'chuckwalla',
 23: 'collared lizard',
 24: 'coyote',
 25: 'desert banded gecko',
 26: 'desert cottontail',
 27: 'desert iguana',
 28: 'desert shrew',
 29: 'desert spiny lizard',
 30: 'desert tortoise',
 31: 'encelia',
 32: 'evening primrose',
 33: 'gray fox',
 34: 'great basin fence lizard',
 35: 'kit fox',
 36: 'leopard lizard',
 37: 'mojave fringe-toed lizard',
 38: 'mountain lion',
 39: 'mule deer',
 40: 'panamint chipmunk',
 41: 'phacelia',
 42: 'poppies',
 43: 'ringtail'

In [0]:
def find_it_predict(model, test_image_path, actual_class, prediction_layer):
  loaded_image = load_image(test_image_path)

  pred = model.predict(loaded_image)
  y_classes=np.argmax(pred)
  prob=max(pred[0])
  img=utils.load_img(test_image_path, target_size=(224, 224))

  plt.imshow(img)
  plt.axis('off')
  print('Prediction: {} (class {}) with a probability of {:.3f}'.format(label_map[y_classes],y_classes,prob))
  print('Actual: {} (class {})'.format(label_map[actual_class],actual_class))

In [23]:
%%time
layer_name='dense_7'
layer_index = utils.find_layer_idx(model, layer_name)
# Swap softmax with linear
model.layers[layer_index].activation = activations.linear
#model.layers[layer_index].activation = activations.relu  #doesn't work to correct the predictions
model = utils.apply_modifications(model)

CPU times: user 18.4 s, sys: 360 ms, total: 18.7 s
Wall time: 18.7 s


In [0]:
def find_it_validate(model, test_image_path, actual_class_label, prediction_layer):
  number_true=0
  number_unknown=0 #such as AMERICAN ROBIN which doesn't exist in MobileNetData folder
  number_wrong=0
  i=0
  
  for file in test_image_path:
    loaded_image = load_image(file)
    pred = model.predict(loaded_image)
    y_classes=np.argmax(pred)
    prob=max(pred[0])

    print('File: {}\nPrediction: {}, probability of {:.3f}, \nactual:     {},'.format(file,label_map[y_classes],prob,actual_class_label[i]))
    if label_map[y_classes]==actual_class_label[i]:
      number_true=number_true+ 1
    else:
      number_wrong=number_wrong+1
    if actual_class_label[i] not in label_map_rev: 
      number_unknown=number_unknown+1
    i=i+1
  print('\nImages checked: {}\nCorrectly predicted: {}\nNumber wrong: {}\nNumber clases not available in training: {}\n% accuracy: {}'.format(len(test_files_class),
                                                                                                                     number_true, number_wrong,number_unknown,number_true/len(test_files_class)))

In [0]:
#define list of test files and actual classes
import os
test_files=[os.path.join(path, name) for path, subdirs, files in os.walk(gpath+'test/') for name in files]
test_classes=[i.split('/')[-2] for i in test_files]
test_files_class=list(zip(test_files,test_classes))

In [0]:
###data to enter in prediction and visualization code
layer_name='dense_7'
layer_index = utils.find_layer_idx(model, layer_name)

In [27]:
find_it_validate(model, test_files,test_classes,layer_name)

File: /content/gdrive/My Drive/Findit/Death Valley/test/AMERICAN PIPIT/American-Pipit-1023x494.jpg
Prediction: BENDIRES THRASHER, probability of 5.263, 
actual:     AMERICAN PIPIT,
File: /content/gdrive/My Drive/Findit/Death Valley/test/AMERICAN PIPIT/images.jpg
Prediction: bobcat, probability of 3.973, 
actual:     AMERICAN PIPIT,
File: /content/gdrive/My Drive/Findit/Death Valley/test/AMERICAN PIPIT/images (1).jpg
Prediction: roadrunner, probability of 4.389, 
actual:     AMERICAN PIPIT,
File: /content/gdrive/My Drive/Findit/Death Valley/test/AMERICAN PIPIT/AmericanPipitIMG_9119.jpg
Prediction: BENDIRES THRASHER, probability of 5.775, 
actual:     AMERICAN PIPIT,
File: /content/gdrive/My Drive/Findit/Death Valley/test/bobcat/7e401868-91cd-4304-8b0b-5e0caedbbc6b-hunting-furbearers-bobcat-NMDGF-New-Mexico.jpg
Prediction: desert cottontail, probability of 5.398, 
actual:     bobcat,
File: /content/gdrive/My Drive/Findit/Death Valley/test/bobcat/images (2).jpg
Prediction: desert iguana, 

In [28]:
#MissingClasses
missing_classes_check= train_datagen.flow_from_directory(gpath+'test', batch_size=512, seed=1234)
missing_classes_check = (missing_classes_check.class_indices)
for i in missing_classes_check:
  if i not in label_map_rev:
    print(i)

Found 57 images belonging to 12 classes.
